In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Cào dữ liệu id của sản phẩm trên từng trang danh mục sản phẩm

In [ ]:
import requests
import time
import random
import pandas as pd

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi-VN,vi;q=0.9',
    'Referer': 'https://tiki.vn/',
    'x-guest-token': 'lrB92vPZFnufEDH0G7jeCxIg5toOkmNM',
    'Connection': 'keep-alive',
    'TE': 'Trailers',
}

In [ ]:
categories = [
    {"urlKey": "nha-sach-tiki", "category": "8322"},
    {"urlKey": "nha-cua-doi-song", "category": "1883"},
    {"urlKey": "dien-thoai-may-tinh-bang", "category": "1789"},
    {"urlKey": "do-choi-me-be", "category": "2549"},
    {"urlKey": "thiet-bi-kts-phu-kien-so", "category": "1815"},
    {"urlKey": "dien-gia-dung", "category": "1882"},
    {"urlKey": "lam-dep-suc-khoe", "category": "1520"},
    {"urlKey": "o-to-xe-may-xe-dap", "category": "8594"},
    {"urlKey": "thoi-trang-nu", "category": "931"},
    {"urlKey": "bach-hoa-online", "category": "4384"},
    {"urlKey": "the-thao-da-ngoai", "category": "1975"},
    {"urlKey": "thoi-trang-nam", "category": "915"},
    {"urlKey": "cross-border-hang-quoc-te", "category": "17166"},
    {"urlKey": "laptop-may-vi-tinh-linh-kien", "category": "1846"},
    {"urlKey": "giay-dep-nam", "category": "1686"},
    {"urlKey": "dien-tu-dien-lanh", "category": "4221"},
    {"urlKey": "giay-dep-nu", "category": "1703"},
    {"urlKey": "may-anh", "category": "1801"},
    {"urlKey": "phu-kien-thoi-trang", "category": "27498"},
    {"urlKey": "ngon", "category": "44792"},
    {"urlKey": "dong-ho-va-trang-suc", "category": "8371"},
    {"urlKey": "balo-va-vali", "category": "6000"},
    {"urlKey": "voucher-dich-vu", "category": "11312"},
    {"urlKey": "tui-vi-nu", "category": "976"},
    {"urlKey": "tui-thoi-trang-nam", "category": "27616"},
    {"urlKey": "cham-soc-nha-cua", "category": "15078"},
]

In [ ]:
all_product_ids = []

In [ ]:
for cat in categories:
    print(f"📦 Đang cào danh mục: {cat['urlKey']} (category={cat['category']})")

    for page in range(1, 10):  # 10 trang đầu
        params = {
            'limit': '40',
            'include': 'advertisement',
            'aggregations': '2',
            'trackity_id': '3732d167-7435-b3b2-dae8-e43bc025db7d',
            'category': cat['category'],
            'page': str(page),
            'src': f"c{cat['category']}",
            'urlKey': cat['urlKey'],
        }

        try:
            response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings',
                                    headers=headers, params=params)
            if response.status_code == 200:
                print(f"✅ Trang {page} OK")
                data = response.json().get('data', [])
                for item in data:
                    all_product_ids.append({
                        "category": cat['urlKey'],
                        "product_id": item.get("id")
                    })
            else:
                print(f"❌ Lỗi tại trang {page} - Mã {response.status_code}")
        except Exception as e:
            print(f"🚨 Lỗi kết nối tại trang {page}: {e}")

        time.sleep(0.3)

In [ ]:
df = pd.DataFrame(all_product_ids)
df.to_csv('output/tiki_product_ids.csv', index=False, encoding='utf-8-sig')
print("🎉 Hoàn tất! Dữ liệu đã lưu vào tiki_product_ids.csv")

#Cào dữ liệu sản phẩm theo id đã cào

In [ ]:
import pandas as pd
import requests
import time
import random
from tqdm import tqdm
from datetime import datetime
import os

In [ ]:
# File output
output_file = 'output/crawled_data.csv'

In [ ]:
# Đọc file CSV chứa category và product_id
df_id = pd.read_csv('tiki_product_ids.csv')
product_list = df_id.to_dict(orient='records')

In [ ]:
if not os.path.exists(output_file):
    pd.DataFrame(columns=[
        'id', 'name', 'da_ban','sku', 'short_description', 'price', 'list_price', 'price_usd',
        'discount', 'discount_rate', 'review_count', 'order_count', 'inventory_status',
        'is_visible', 'stock_item_qty', 'stock_item_max_sale_qty', 'product_name',
        'brand_id', 'brand_name', 'category', 'crawl_date'
    ]).to_csv(output_file, index=False)

In [ ]:
cookies = {
    'TIKI_GUEST_TOKEN': '8jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY',
    'TOKENS': '{%22access_token%22:%22lrB92vPZFnufEDH0G7jeCxIg5toOkmNM%22%2C%22expires_in%22:157680000%2C%22expires_at%22:1902390739109%2C%22guest_token%22:%22lrB92vPZFnufEDH0G7jeCxIg5toOkmNM%22}',
    'amp_99d374': 'eSc-_0HT1um7cb57E7dwA0...1enloc6a2.1enlohtdv.3.2.5',
    'amp_99d374_tiki.vn': 'eSc-_0HT1um7cb57E7dwA0...1enloc6a2.1enlocds8.0.1.1',
    '_gcl_au': '1.1.559117409.1605974236',
    '_ants_utm_v2': '',
    '_pk_id.638735871.2fc5': 'b92ae025fbbdb31f.1605974236.1.1605974420.1605974236.',
    '_pk_ses.638735871.2fc5': '*',
    '_trackity': '70e316b0-96f2-dbe1-a2ed-43ff60419991',
    '_ga_NKX31X43RV': 'GS1.1.1605974235.1.1.1605974434.0',
    '_ga': 'GA1.1.657946765.1605974236',
    'ai_client_id': '11935756853.1605974227',
    'an_session': 'zizkzrzjzlzizqzlzqzjzdzizizqzgzmzkzmzlzrzmzgzdzizlzjzmzqzkznzhzhzkzdzhzdzizlzjzmzqzkznzhzhzkzdzizlzjzmzqzkznzhzhzkzdzjzdzhzqzdzizd2f27zdzjzdzlzmzmznzq',
    'au_aid': '11935756853',
    'dgs': '1605974411%3A3%3A0',
    'au_gt': '1605974227146',
    '_ants_services': '%5B%22cuid%22%5D',
    '__admUTMtime': '1605974236',
    '__iid': '749',
    '__su': '0',
    '_bs': 'bb9a32f6-ab13-ce80-92d6-57fd3fd6e4c8',
    '_gid': 'GA1.2.867846791.1605974237',
    '_fbp': 'fb.1.1605974237134.1297408816',
    '_hjid': 'f152cf33-7323-4410-b9ae-79f6622ebc48',
    '_hjFirstSeen': '1',
    '_hjIncludedInPageviewSample': '1',
    '_hjAbsoluteSessionInProgress': '0',
    '_hjIncludedInSessionSample': '1',
    'tiki_client_id': '657946765.1605974236',
    '__gads': 'ID=ae56424189ecccbe-227eb8e1d6c400a8:T=1605974229:RT=1605974229:S=ALNI_MZFWYf2BAjzCSiRNLC3bKI-W_7YHA',
    'proxy_s_sv': '1605976041662',
    'TKSESSID': '8bcd49b02e1e16aa1cdb795c54d7b460',
    'TIKI_RECOMMENDATION': '21dd50e7f7c194df673ea3b717459249',
    '_gat': '1',
    'cto_bundle': 'i6f48l9NVXNkQmJ6aEVLcXNqbHdjcVZoQ0k2clladUF2N2xjZzJ1cjR6WG43UTVaRmglMkZXWUdtRnJTNHZRbmQ4SDAlMkZwRFhqQnppRHFxJTJCSEozZXBqRFM4ZHVxUjQ2TmklMkJIcnhJd3luZXpJSnBpcE1nJTNE',
    'TIKI_RECENTLYVIEWED': '58259141',
}

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi-VN,vi;q=0.8,en-US;q=0.5,en;q=0.3',
    'Referer': 'https://tiki.vn/',
    'x-guest-token': 'lrB92vPZFnufEDH0G7jeCxIg5toOkmNM',
    'Connection': 'keep-alive',
}

In [ ]:
params = (('platform', 'web'),)

In [ ]:
def parser_product(json):
    d = dict()
    d['id'] = json.get('id')
    d['name'] = json.get('name')
    d['daban'] = json.get('all_time_quantity_sold')
    d['sku'] = json.get('sku')
    d['short_description'] = json.get('short_description')
    d['price'] = json.get('price')
    d['list_price'] = json.get('list_price')
    d['price_usd'] = json.get('price_usd')
    d['discount'] = json.get('discount')
    d['discount_rate'] = json.get('discount_rate')
    d['review_count'] = json.get('review_count')
    d['order_count'] = json.get('order_count')
    d['inventory_status'] = json.get('inventory_status')
    d['is_visible'] = json.get('is_visible')
    stock = json.get('stock_item')
    d['stock_item_qty'] = stock.get('qty') if stock else None
    d['stock_item_max_sale_qty'] = stock.get('max_sale_qty') if stock else None
    d['product_name'] = json.get('meta_title')
    brand = json.get('brand')
    d['brand_id'] = brand.get('id') if brand else None
    d['brand_name'] = brand.get('name') if brand else None
    return d


In [ ]:
# Danh sách kết quả
result = []
crawl_date = datetime.now().strftime('%Y-%m-%d')

for item in tqdm(product_list, total=len(product_list)):
    pid = item['product_id']
    category = item['category']

    response = requests.get(f'https://tiki.vn/api/v2/products/{pid}', headers=headers, params=params, cookies=cookies)
    if response.status_code == 200:
        print(f'Crawl data {pid} success !!!')
        data = parser_product(response.json())
        # Thêm category, id, crawl_date vào mỗi dòng
        data['category'] = category
        data['crawl_date'] = crawl_date
        result.append(data)

        pd.DataFrame([data]).to_csv(output_file, mode='a', header=False, index=False)
        #pd.DataFrame([data]).to_csv(output_file1, mode='a', header=False, index=False)
    else:
        print(f'❌ Fail to crawl product {pid}')
    time.sleep(0.7)

#Cào dữ liệu bình luận đánh giá của khách hàng trên các id sản phẩm

In [ ]:
import requests
import pandas as pd
import time
import random
import os
from tqdm import tqdm

In [ ]:
output_file = 'comments_data_ncds.csv'

if not os.path.exists(output_file):
    pd.DataFrame(columns=[
        'id', 'title', 'content','thank_count', 'customer_id', 'rating', 'created_at', 'customer_name',
        'purchased_at'
    ]).to_csv(output_file, index=False)

In [ ]:
cookies = {
    'TIKI_GUEST_TOKEN': '8jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY',
    'TOKENS': '{%22access_token%22:%228jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY%22%2C%22expires_in%22:157680000%2C%22expires_at%22:1763654224277%2C%22guest_token%22:%228jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY%22}',
    'amp_99d374': 'eSc-_0HT1um7cb57E7dwA0...1enloc6a2.1enlrj4bc.1k.11.2l',
    'amp_99d374_tiki.vn': 'eSc-_0HT1um7cb57E7dwA0...1enloc6a2.1enlrj2q9.3.1.1',
    '_gcl_au': '1.1.559117409.1605974236',
    '_ants_utm_v2': '',
    '_pk_id.638735871.2fc5': 'b92ae025fbbdb31f.1605974236.1.1605977607.1605974236.',
    '_pk_ses.638735871.2fc5': '*',
    '_trackity': '70e316b0-96f2-dbe1-a2ed-43ff60419991',
    '_ga_NKX31X43RV': 'GS1.1.1605974235.1.1.1605977607.0',
    '_ga': 'GA1.2.657946765.1605974236',
    'ai_client_id': '11935756853.1605974227',
    'an_session': 'zizkzrzjzkzizhzkzlznzdzizizqzgzmzkzmzlzrzmzgzdzizlzjzmzqzkzlzjzgzjzdzizizdzizlzjzmzqzkzlzjzgzjzdzizlzjzmzqzkzlzjzgzjzdzjzdzhzqzdzizd2f27zdzjzdzlzmzmznzq',
    'au_aid': '11935756853',
    'dgs': '1605977598%3A3%3A0',
    'au_gt': '1605974227146',
    '_ants_services': '%5B%22cuid%22%5D',
    '__admUTMtime': '1605974236',
    '__iid': '749',
    '__su': '0',
    '_bs': 'bb9a32f6-ab13-ce80-92d6-57fd3fd6e4c8',
    '_gid': 'GA1.2.867846791.1605974237',
    '_fbp': 'fb.1.1605974237134.1297408816',
    '_hjid': 'f152cf33-7323-4410-b9ae-79f6622ebc48',
    '_hjFirstSeen': '1',
    '_hjAbsoluteSessionInProgress': '0',
    'tiki_client_id': '657946765.1605974236',
    '__gads': 'ID=ae56424189ecccbe-227eb8e1d6c400a8:T=1605974229:RT=1605974229:S=ALNI_MZFWYf2BAjzCSiRNLC3bKI-W_7YHA',
    'proxy_s_sv': '1605978058486',
    'TKSESSID': '8bcd49b02e1e16aa1cdb795c54d7b460',
    'TIKI_RECOMMENDATION': '21dd50e7f7c194df673ea3b717459249',
    'cto_bundle': '7L6ha19NVXNkQmJ6aEVLcXNqbHdjcVZoQ0kzTUZwcEMyNCUyRm5nV3A2SThuOGxTRjI4Wlk1NU9xRnBEOG9tUjd2ekhyZEQxeE9qaVQ4MnFpbiUyRllGd2JiQUpTMW94MlNsTnYxd3dOYWtRcXhGdDNxSjdBVmNxU0FnUSUyQjlWYjhqTUtLdVl2cTBheWFvS0ZnY2pLdlpWRlEyUFF0Y1ElM0QlM0Q',
    'TIKI_RECENTLYVIEWED': '58259141',
    '_ants_event_his': '%7B%22action%22%3A%22view%22%2C%22time%22%3A1605977607258%7D',
    '_hjIncludedInPageviewSample': '1',
    '_hjIncludedInSessionSample': '1',
    '_gat': '1',
}

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi-VN,vi;q=0.8,en-US;q=0.5,en;q=0.3',
    'Referer': 'https://tiki.vn/dien-thoai-samsung-galaxy-m31-128gb-6gb-hang-chinh-hang-p58259141.html',
    'x-guest-token': '8jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY',
    'Connection': 'keep-alive',
    'TE': 'Trailers',
}

In [ ]:
params = {
    'product_id': '58259141',
    'sort': 'score|desc,id|desc,stars|all',
    'page': '1',
    'limit': '10',
    'include': 'comments'
}

In [ ]:
def comment_parser(json):
    d = dict()
    d['id'] = json.get('id')
    d['title'] = json.get('title')
    d['content'] = json.get('content')
    d['thank_count'] = json.get('thank_count')
    d['customer_id']  = json.get('customer_id')
    d['rating'] = json.get('rating')
    d['created_at'] = json.get('created_at')
    d['customer_name'] = json.get('created_by').get('name')
    d['purchased_at'] = json.get('created_by').get('purchased_at')
    return d

In [ ]:
df_id = pd.read_csv('product_id_ncds.csv')
print(df_id.head())
p_ids = df_id.id.to_list()
result = []
for pid in tqdm(p_ids, total=len(p_ids)):
    params['product_id'] = pid
    print('Crawl comment for product {}'.format(pid))
    for i in range(1):
        params['page'] = i
        response = requests.get('https://tiki.vn/api/v2/reviews', headers=headers, params=params, cookies=cookies)
        if response.status_code == 200:
            print('Crawl comment page {} success!!!'.format(i))
            for comment in response.json().get('data'):
                result.append(comment_parser(comment))
                pd.DataFrame(result).to_csv(output_file, mode='a', header=False, index=False)
        time.sleep(0.3)